# PhD proposal - Matteo Pompili
$\newcommand{\ket}[1]{|{#1}\rangle} \newcommand{\bra}[1]{\langle{#1}|} \newcommand{\braket}[1]{\langle{#1}\rangle} \newcommand{\ketbra}[1]{\ket{#1}\bra{#1}}$

## Research goal

The goal of my PhD is: **Demonstration of quantum applications on a multi-node network**

## Achieved so far

### Experimentally

So far I built a new setup (LT5) that will act as one of the nodes in three-node experiment.
By the end of November we should be able to generate entanglement between LT5 and LT4. 

### Courses

#### January 2018
+ Collaboration across disciplines (? GSC)

#### October 2018  

+ Conversation skills (2 GSC)

#### November 2018

+ Casimir Course - Programming (5 GSC)
+ The Art of Presenting Science (5 GSC)
+ QuTech Academy - Quantum Communication and Cryptography (5 GSC)

#### December 2018

+ Casimir Course - Electronics for Physicists (5 GSC)

### Supervision

I have been supervising Hans K. C. Beukers, a MSc student, since February. He has been working on improvements that, if successful, will increase the lifetime of our memory qubits.

### Outreach

I carried out two outreach activities within my MSCA network (*Spin-NANO*):

+ January 2018, Sheffield, UK. Introduction to quantum- and nano-technologies to local high-school students.
+ September 2018, Brussels, BE. Two days stand about quantum technologies at the ERN, EU Parlamentarium, mainly to children between 5 and 10.

## Plan

#### 2019

*Demonstrate the generation of a 3-node GHZ state* and use it to test violation of an appropriate inequality.
One of the nodes will use a memory qubit. (First half of 2019)

To be able to carry out complex experiments on a 3+ node network with NV centres we need to overhaul our classical controls. 
In particular, our software infrastructure, although very powerful, has not been updated in years. As a consequence implementing the new experiments would become extremely challenging. 
*I therefore plan to rewrite the needed software in a modern framework.* 
I will use the framework that is being developed for the QINC-demonstrator (qmi) and adapt and update our old code to run on this new platform. (Second half / third quarter of 2019)

#### 2020

*Build 4th node and demonstrate deterministic teleportation of entanglement* as proposed in the QNETWORK proposal. To achieve this we will need memory qubits on at least two nodes.

note: Simon is probably leaving @ 3/4 of this year.

#### 2021

*Write thesis*

### Violation of CHSH inequalities for 3-node GHZ state

<img src="images/sketch_three_nodes.jpg" width="50%">

Let's try to estimate the fidelity of the expected GHZ state. It is likely that we will run the first and second entanglment with different rates $\alpha$, that is $\alpha_1$ and $\alpha_2$.

The first entangled state, generated between *Alice* and *Charlie*, will run with $\alpha_1$. We will probably prefer to run with a low rate here, to maximize fidelity, such that we can run faster with the second one (and lose less on dechoerence of the first entangled state). Let's say we choose $\alpha_1 = \max_{F_{AC}} \alpha$, where $F_{AC}$ is the maximum fidelity of the entangled state between A and C. Dark counts on the ZPL detectors limit the minimum rate, so there will be an $\alpha_1 > 0$ s.t. fidelity is optimized.

If one assumes no dark counts and no other imperfections in the entanglement generation the fidelity of the state goes like $F(\alpha) = (1-\alpha)$

In [1]:
import numpy as np
import qutip as qt
import matplotlib.pyplot as plt

plt.style.use('ggplot')

def fidelity(*args):
    return qt.fidelity(*args)**2

alfas = np.linspace(0,1)
fids = np.zeros_like(alfas)
for i, x in enumerate(alfas):
    updown = qt.tensor(qt.basis(2,1), qt.basis(2,0))
    downup = qt.tensor(qt.basis(2,0), qt.basis(2,1))
    upup = qt.tensor(qt.basis(2,1), qt.basis(2,1))
    psi_ac = (updown + downup).unit()
    rho_ac = (1-x) * qt.ket2dm(psi_ac) + (x) * qt.ket2dm(upup)
    fids[i] = fidelity(rho_ac, qt.ket2dm(psi_ac))
    
plt.title('Fidelity without experimental imperfections')
plt.xlabel(r'Bright state population $\alpha$')
plt.ylabel(r'Fidelity with $|\Psi\rangle$')
plt.scatter(alfas, fids)
plt.show()

<Figure size 640x480 with 1 Axes>

At this point the entangled state between A and C can be written as $\rho_{AC} = f(F_{AC}) \ketbra{\Psi_{AC}} + (1-f(F_{AC})) \frac{\hat{\mathbb{I}}}{4}$, where $f(F_{AC})$ is a number s.t. the fidelity of $\rho_{AC}$ with $\ketbra{\Psi_{AC}}$ is $F_{AC}$. For example, if the fidelity is 0.80, then $f(\alpha_1) \approx 0.73$.

In [2]:
f=0.73
downup = qt.tensor(qt.basis(2,0), qt.basis(2,1))
updown = qt.tensor(qt.basis(2,1), qt.basis(2,0))
psi_ac = (updown + downup).unit()
mixed_state = qt.tensor(qt.maximally_mixed_dm(2), qt.maximally_mixed_dm(2))
rho_ac = f * qt.ket2dm(psi_ac) + (1-f) * mixed_state
print('Fidelity with f={} is: {:.2f}'.format(f, fidelity(rho_ac, qt.ket2dm(psi_ac))))

Fidelity with f=0.73 is: 0.80


At this point we perform our SWAP gate. The operation will have some fidelity with the ideal SWAP gate. We can model the final state as swapped with probability $p_{SWAP}$ and a mixed carbon state and mixed electron state with probability $1 - p_{SWAP}$. In the distillation paper $p_{SWAP} = 0.98$, but it needs to be combined with the probability of not flipping the electron upon reading (also $\approx 0.98$). Let's write the whole process as $p_{SWAP} = 0.95$ success probability.
$\rho_{Ac} = p_{SWAP} (f\ketbra{\Psi} + (1-f)\frac{\hat{\mathbb{I}}}{4}) + (1-p_{SWAP})tr_C[\rho_{AC}] \frac{\hat{\mathbb{I_c}}}{4}$.

Let's make a quick test with the 0.8 fidelity of before and the swap gate, and let's check the fidelity afterwards.

In [3]:
p_swap = 0.95
el_el_state = rho_ac
el_carbon_state = p_swap * rho_ac + (1 - p_swap) * qt.tensor(rho_ac.ptrace(0), qt.maximally_mixed_dm(2))
print('Fidelity of the state after the swap is: {:.2f}'.format(fidelity(el_carbon_state, qt.ket2dm(psi_ac))))

Fidelity of the state after the swap is: 0.77


At this point we want to generate entanglement again. The electron carbon state will dechoere as times passes. Off diagonal elements will decrese exponentially.
In the distillation paper it looks like the number of cycles after which the state dechoered of $1/e$ is 270 ($\approx 2ms$). At 1/e the fidelity of the entangles state is already lower than the classically correlated state. Therefore we either need to increase the coherence time, or we need to generate entanglement faster.

In [4]:
def dechoere(m, t, t0):
    data = m.full()
    diag_m = np.diagflat(m.diag())
    data = diag_m + (data - diag_m) * np.exp(-t/t0)
    return qt.Qobj(data, dims=m.dims)

In [5]:
cycles_to_entangle = 200
cycles_of_coherence = 270
fid = fidelity(qt.ket2dm(psi_ac), dechoere(el_carbon_state, cycles_to_entangle, cycles_of_coherence))
print('Fidelity with the state after {} attempts is: {:.2f}'.format(cycles_to_entangle, fid))

Fidelity with the state after 200 attempts is: 0.59


The new entangled state can be generate faster with lower fidelity than the first. Let's try to write down the whole thing.
Qubit 1 is electron_alice. Qubit 2 is electron_bob. Qubit 3 is carbon_bob. Qubit 4 is electron_charlie.
We assume the fidelity-rate curves of deterministic entanglement (altought we could be able to do better, by getting rid of excitation light in the ZPL).
My matching it with the exp data it looks like the following function might do.

In [28]:
def alpha_to_fidelity(alpha, dc = 0.01, top_fid = 0.9):
    return (1 - np.exp(-alpha/dc)) * (top_fid - alpha)

def alpha_to_rate(alpha, speed = 125):
    return alpha * speed

import scipy as sp

opt_res = sp.optimize.minimize_scalar(lambda x: - alpha_to_fidelity(x), bounds=(0,1))
slow_alpha = opt_res.x
print ('Max fidelity achievable with old parameters is {:.3f}, at alpha = {:.3f}, which corresponds to a rate = {:.1f} Hz'\
       .format(alpha_to_fidelity(slow_alpha),
               slow_alpha, 
               alpha_to_rate(slow_alpha)))

Max fidelity achievable with old parameters is 0.846, at alpha = 0.045, which corresponds to a rate = 5.6 Hz


In [29]:
# Generate entangled state with max fidelity
q1q4_ent_state = alpha_to_fidelity(slow_alpha) * qt.ket2dm(psi_ac) + (1- alpha_to_fidelity(slow_alpha)) * qt.ket2dm(upup)

# Apply swap gate or project in mixed state
p_swap = 0.95
q1q3_ent_state = p_swap * q1q4_ent_state + (1 - p_swap) * mixed_state

print('Now fidelity with Bell state is {}'.format(fidelity(q1q3_ent_state, qt.ket2dm(psi_ac))))

Now fidelity with Bell state is 0.81573408796723


In [89]:
# Now we generate entangled state again with lower alpha and we dechoere the first one according to rate

fast_alpha = 0.2
fast_rate = alpha_to_rate(fast_alpha)
slow_rate = alpha_to_rate(slow_alpha)
fast_cycles = 200 # 1. / fast_rate / 5.5E-6
prob_generation_before_timeout = 1 - np.exp(- fast_rate * 5.5E-6 * fast_cycles) 

cycles_of_coherence = 1000
decohered_q1q3_ent_state = dechoere(q1q3_ent_state, t=fast_cycles, t0=cycles_of_coherence)

q2q4_ent_state = alpha_to_fidelity(fast_alpha) * qt.ket2dm(psi_ac) + (1- alpha_to_fidelity(fast_alpha)) * qt.ket2dm(upup)

joint_state = qt.tensor(decohered_q1q3_ent_state, q2q4_ent_state)

# And let's reorder the tensor such that we much the previous defintion of qubits.
joint_state = joint_state.permute([0,2,1,3])
print('Fidelity of q1-q3 to bell state: {:.2f}'.format(fidelity(joint_state.ptrace([0,2]), qt.ket2dm(psi_ac))))
print('Fidelity of q2-q4 to bell state: {:.2f}'.format(fidelity(joint_state.ptrace([1,3]), qt.ket2dm(psi_ac))))
print('Probability of getting the state before timeout: {:.2f}'.format(prob_generation_before_timeout))
print('Overall generation rate: {:1f} Hz'.format((1/fast_rate + 1/slow_rate) * (prob_generation_before_timeout)))
print('Overall generation time: {:1f} s'.format(1/((1/fast_rate + 1/slow_rate) * (prob_generation_before_timeout))))

Fidelity of q1-q3 to bell state: 0.74
Fidelity of q2-q4 to bell state: 0.70
Probability of getting the state before timeout: 0.03
Overall generation rate: 0.005950 Hz
Overall generation time: 168.070442 s


Now we want to apply our conditional rotations to entangle q3 with q4 (carbon and electron of Charlie) and then disentangle q4 by measurement and feedback.

<img src="images/cond_gate_el_ca.png" width="20%">

In [90]:
q4_xmpi2 = qt.rotation(qt.sigmax(), -np.pi/2, 4, 3)
q4_xppi2 = qt.rotation(qt.sigmax(), +np.pi/2, 4, 3)
controlled_rot1 = qt.controlled_gate(qt.rotation(qt.sigmax(), np.pi/2), N=4, control=3, target=2, control_value=0)
controlled_rot2 = qt.controlled_gate(qt.rotation(qt.sigmax(), -np.pi/2), N=4, control=3, target=2, control_value=1)
elec_carb_gate = q4_xppi2 * controlled_rot2 * controlled_rot1 * q4_xmpi2

joint_state = elec_carb_gate * joint_state * elec_carb_gate.dag()

Now we can apply our classical feedback after measuring q4. If it is in zero we don't do anything.

In [91]:
q4_zero_proj = qt.tensor(qt.identity(2), qt.identity(2), qt.identity(2), qt.basis(2, 0).proj())
q4_one_proj = qt.tensor(qt.identity(2), qt.identity(2), qt.identity(2), qt.basis(2, 1).proj())

classical_feedback = q4_zero_proj + q4_one_proj*qt.rotation(qt.sigmax(), np.pi, 4, 0)*qt.rotation(qt.sigmaz(), np.pi, 4, 1)
final_state = (classical_feedback * joint_state * classical_feedback.dag()).ptrace([0,1,2])

In [92]:
zero = qt.basis(2, 0)
one  = qt.basis(2, 1)
s000 = qt.tensor(zero, zero, zero)
s110 = qt.tensor(one, one, zero)
s101 = qt.tensor(one, zero, one)
s011 = qt.tensor(zero, one, one)
target_state = (s000 + s110 + s101 + s011).unit()

print('Fidelity of GHZ-like final state with target state: {:0.3f}'.format(fidelity(final_state, target_state)))

Fidelity of GHZ-like final state with target state: 0.533
